In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("training_data.csv")
test_df  = pd.read_csv("test_data.csv")


In [ ]:
X = train_df.drop(columns=["target"])
y = train_df["target"]

X_test = test_df.copy()


In [ ]:
N_FOLDS = 5

lgb_oof = np.zeros(len(X))          # out-of-fold predictions
lgb_test_folds = np.zeros((len(X_test), N_FOLDS))  # test preds per fold


In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=N_FOLDS,
    shuffle=True,
    random_state=42
)

In [ ]:
import lightgbm as lgb
pos=(y==1).sum()
neg=(y==0).sum()
ratio=neg/pos
print(ratio)

26.436992221261885


In [ ]:
model_params = {
    "objective": "binary",
    "scale_pos_weight": ratio,
    "learning_rate": 0.01,           # ← increased from 0.01
    "num_leaves": 31,                # ← increased from 31
    "min_data_in_leaf": 200,          # ← decreased from 200
    "n_estimators": 800,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 3,
    "reg_alpha": 0.5,
    "reg_lambda": 1.0,
    "random_state": 42,
    "n_jobs": -1
}


In [ ]:
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.metrics import roc_auc_score

lgb_models = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    X_train = X.iloc[train_idx].drop(columns=["id"])
    X_val   = X.iloc[val_idx].drop(columns=["id"])

    y_train = y.iloc[train_idx]
    y_val   = y.iloc[val_idx]

    model = LGBMClassifier(**model_params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[log_evaluation(100)]
    )

    lgb_models.append(model)

    val_preds = model.predict_proba(X_val)[:, 1]
    lgb_oof[val_idx] = val_preds

    lgb_test_folds[:, fold] = model.predict_proba(
        X_test.drop(columns=["id"])
    )[:, 1]

    fold_auc = roc_auc_score(y_val, val_preds)
    print(f"Fold {fold} AUC: {fold_auc:.4f}")


[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 13884, number of

In [ ]:
auc = roc_auc_score(y, lgb_oof)
gini = 2 * auc - 1

print("CV AUC:", auc)
print("CV Gini:", gini)


CV AUC: 0.6401863502376878
CV Gini: 0.28037270047537555


In [ ]:
X_test_new = pd.read_csv("test_data.csv")   # or whatever the file name is
test_ids = X_test_new["id"]            # if id column exists

X_test_new = X_test_new.drop(columns=["id"], errors="ignore")


In [ ]:
print(X_train.shape)
print(X_test_new.shape)

print(set(X_train.columns) - set(X_test_new.columns))
print(set(X_test_new.columns) - set(X_train.columns))



(380936, 50)
(119043, 50)
set()
set()


In [ ]:
X_test_new = X_test_new[X_train.columns]


In [ ]:
test_probs = np.zeros(len(X_test_new))

for model in lgb_models:  # list of trained fold models
    test_probs += model.predict_proba(X_test_new)[:, 1]

test_probs /= len(lgb_models)

[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_

In [ ]:
print(test_probs)

[0.36450181 0.57505288 0.2895587  ... 0.66674672 0.50072682 0.33083109]


In [ ]:
submission_lgb = pd.DataFrame({
    "id": test_ids,
    "target": test_probs
})

submission_lgb.to_csv("submission_lgb_raw.csv", index=False)


In [ ]:
lgb_test = lgb_test_folds.mean(axis=1)


In [ ]:
pd.DataFrame({
    "row_idx": np.arange(len(lgb_test)),
    "lgb_test": lgb_test
}).to_csv("/content/lgb_test.csv", index=False)


In [ ]:
pd.DataFrame({
    "row_idx": np.arange(len(lgb_oof)),
    "lgb_oof":lgb_oof
}).to_csv("lgb_oof.csv", index=False)
